In [5]:
import numpy as np
import matplotlib.pyplot as plt
import aipy as a 
import capo
import capo.miriad as miriad
from scipy.sparse import csr_matrix
from tqdm import tqdm
import capo.omni as omni
from copy import deepcopy
import capo.linsolve
#import seaborn
%matplotlib notebook 

# Array Code

In [87]:
class InterferometricArray():
    """Class that takes a list of positions and can calcuate baselines and redundancies."""
    
    def __init__(self, positions=[]):
        self.positions = np.array(positions)
        self.nant = len(positions)
        self.antNames = range(self.nant)
    
    def CalculateUBLs(self, precisionFactor=1000000):
        """Finds the baselines, unique baselines, and related dictionaries for indexing."""
        self.blVectors, self.blNamePairs, self.blIndexPairs = [], [], []
        self.index2name = {i:name for i,name in enumerate(self.antNames)}
        self.name2index = {name:i for i,name in enumerate(self.antNames)}
        self.name2pos = {name: self.positions[self.name2index[name]] for name in self.antNames}
        for index1,ant1 in enumerate(self.antNames):
            for index2,ant2 in zip(range(index1+1,self.nant), self.antNames[index1+1:]):
                delta = np.array([int(np.round(precisionFactor*(self.positions[index1][i] - self.positions[index2][i]))) for i in range(3)])
                if delta[1] > 0 or (delta[1] == 0 and delta[0] > 0): 
                    self.blVectors.append(tuple(delta))
                    self.blNamePairs.append((ant1, ant2))
                    self.blIndexPairs.append((index1, index2))
                else: 
                    self.blVectors.append(tuple(-delta))
                    self.blNamePairs.append((ant2, ant1))
                    self.blIndexPairs.append((index2, index1))
        self.ublDict = {}
        for b in range(len(self.blVectors)):
            if self.ublDict.has_key(self.blVectors[b]): self.ublDict[self.blVectors[b]].append(self.blNamePairs[b])
            else: self.ublDict[self.blVectors[b]] = [self.blNamePairs[b]]
        self.blIndexDict = {antPair: i for i,antPair in enumerate(self.blNamePairs)}
        self.names2ublIndex = {antPair: i for i,antPairs in enumerate(self.ublDict.values()) for antPair in antPairs}
        self.indices2ublIndex = {(self.name2index[antPair[0]],self.name2index[antPair[1]]): 
                                 i for i,antPairs in enumerate(self.ublDict.values()) for antPair in antPairs}
        self.ublVectors = np.array([self.name2pos[antList[0][0]]-self.name2pos[antList[0][1]] for antList in self.ublDict.values()])
        self.ublGroups = [antList for antList in self.ublDict.values()]
        print "With", len(self.positions), "antennas there are", len(self.ublDict.items()), "unique baselines."
        self.nbl, self.nubl = len(self.blNamePairs), len(self.ublVectors)

class HexagonalArray(InterferometricArray):
    """Generates a hexagonal array."""
    
    def __init__(self, separation, hexNum, verbose=False):
        """Creates a hexagonal array with hexNum antennas per side separated by separation."""
        self.hexNum, self.separation, self.verbose = hexNum, separation, verbose
        positions, self.rowIndices, i = [], [], 0        
        for row in range(hexNum-1,-(hexNum),-1):
            indices = []
            for col in range(2*hexNum-abs(row)-1):
                xPos = ((-(2*hexNum-abs(row))+2)/2.0 + col)*separation;
                yPos = row*separation*3**.5/2;
                positions.append([xPos, yPos, 0])
                indices.append(i); i+=1
            self.rowIndices.append(indices)
        self.positions = np.array(positions) 
        self.nant = len(self.positions)
        self.antNames = range(self.nant)

# Omnical with a single polarization

In [95]:
ha = HexagonalArray(14.7, 3)
ha.CalculateUBLs()

trueVis = {}
for group in ha.ublGroups: 
    trueVis[group[0]] = np.random.randn() + 1.0j*np.random.randn()

trueGains = {}
for ant in ha.antNames:
    trueGains[ant] = 1.0 + .1*(np.random.randn() + np.random.randn())

obsVis, equations = {}, {}
for group in ha.ublGroups: 
    for (i,j) in group:
        obsVis[(i,j)] = trueGains[i] * np.conj(trueGains[j]) * trueVis[group[0]]# + .01*np.random.randn() + .01j*np.random.randn()
        equations['g{} * g{}_ * v{}to{}'.format(i,j,group[0][0], group[0][1])] = obsVis[(i,j)]

logSolver = capo.linsolve.LogProductSolver(equations)
logSol = logSolver.solve()
linSolver = capo.linsolve.LinProductSolver(equations, logSol)
linSol = linSolver.solve()

chiSq = 0
for group in ha.ublGroups: 
    for (i,j) in group:
        chiSq += np.abs(obsVis[(i,j)] - linSol['g{}'.format(i)] * np.conj(linSol['g{}'.format(j)]) 
                        * linSol['v{}to{}'.format(group[0][0], group[0][1])])**2
print 'Chisq:', chiSq

A = linSolver.ls.get_A()
A = A.reshape((A.shape[0],A.shape[1]))
AtA = A.T.dot(A)
print 'Number of omnical degeneracies: ', len(AtA) - np.linalg.matrix_rank(AtA)



With 19 antennas there are 30 unique baselines.
Chisq: 5.96360972319e-30
Number of omnical degeneracies:  4


# Omnical with two independent polarizations ($xx$ and $yy$ only)

In [99]:
ha = HexagonalArray(14.7, 3)
ha.CalculateUBLs()
antpols = ['x','y']
vispols = ['xx','yy']

trueVis = {}
for group in ha.ublGroups: 
    trueVis[group[0]] = {pol: np.random.randn() + 1.0j*np.random.randn() for pol in vispols}

trueGains = {}
for ant in ha.antNames:
    trueGains[ant] = {pol: 1.0 + .1*(np.random.randn() + np.random.randn()) for pol in antpols}

obsVis, equations = {}, {}
for group in ha.ublGroups: 
    for (i,j) in group:
        obsVis[(i,j)] = {pol: trueGains[i][pol[0]] * np.conj(trueGains[j][pol[1]]) * trueVis[group[0]][pol] for pol in vispols}
        for pol in vispols:
            eqStr = 'g{}{} * g{}{}_ * v{}to{}{}'.format(i,pol[0],j,pol[1],group[0][0],group[0][1],pol)
            equations[eqStr] = obsVis[(i,j)][pol]

logSolver = capo.linsolve.LogProductSolver(equations)
logSol = logSolver.solve()
linSolver = capo.linsolve.LinProductSolver(equations, logSol)
linSol = linSolver.solve()

chiSq = 0
for group in ha.ublGroups: 
    for (i,j) in group:
        for pol in vispols:
            chiSq += np.abs(obsVis[(i,j)][pol] - linSol['g{}{}'.format(i,pol[0])] * np.conj(linSol['g{}{}'.format(j,pol[1])])
                            * linSol['v{}to{}{}'.format(group[0][0], group[0][1], pol)])**2
print 'Chisq:', chiSq

A = linSolver.ls.get_A()
A = A.reshape((A.shape[0],A.shape[1]))
AtA = A.T.dot(A)
print 'Number of omnical degeneracies: ', len(AtA) - np.linalg.matrix_rank(AtA)

With 19 antennas there are 30 unique baselines.
Chisq: 1.56184041156e-29
Number of omnical degeneracies:  8


# Omnical with four independent visibility polarizations ($V_{xy} \neq V^*_{yx}$)

In [100]:
ha = HexagonalArray(14.7, 3)
ha.CalculateUBLs()
antpols = ['x','y']
vispols = ['xx','yy','xy','yx']

trueVis = {}
for group in ha.ublGroups: 
    trueVis[group[0]] = {pol: np.random.randn() + 1.0j*np.random.randn() for pol in vispols}

trueGains = {}
for ant in ha.antNames:
    trueGains[ant] = {pol: 1.0 + .01*(np.random.randn() + np.random.randn()) for pol in antpols}

obsVis, equations = {}, {}
for group in ha.ublGroups: 
    for (i,j) in group:
        obsVis[(i,j)] = {pol: trueGains[i][pol[0]] * np.conj(trueGains[j][pol[1]]) * trueVis[group[0]][pol] for pol in vispols}
        for pol in vispols:
            eqStr = 'g{}{} * g{}{}_ * v{}to{}{}'.format(i,pol[0],j,pol[1],group[0][0],group[0][1],pol)
            equations[eqStr] = obsVis[(i,j)][pol]

logSolver = capo.linsolve.LogProductSolver(equations)
logSol = logSolver.solve()
linSolver = capo.linsolve.LinProductSolver(equations, logSol)
linSol = linSolver.solve()

chiSq = 0
for group in ha.ublGroups: 
    for (i,j) in group:
        for pol in vispols:
            chiSq += np.abs(obsVis[(i,j)][pol] - linSol['g{}{}'.format(i,pol[0])] * np.conj(linSol['g{}{}'.format(j,pol[1])])
                            * linSol['v{}to{}{}'.format(group[0][0], group[0][1], pol)])**2
print 'Chisq:', chiSq

A = linSolver.ls.get_A()
A = A.reshape((A.shape[0],A.shape[1]))
AtA = A.T.dot(A)
print 'Number of omnical degeneracies: ', len(AtA) - np.linalg.matrix_rank(AtA)

With 19 antennas there are 30 unique baselines.
Chisq: 4.00161538643e-29
Number of omnical degeneracies:  6


# Omnical with four independent visibility polarizations ($V_{xy} \equiv V^*_{yx}$)

In [104]:
ha = HexagonalArray(14.7, 3)
ha.CalculateUBLs()
antpols = ['x','y']
vispols = ['xx','yy','xy','yx']

trueVis = {}
for group in ha.ublGroups: 
    trueVis[group[0]] = {pol: np.random.randn() + 1.0j*np.random.randn() for pol in vispols}
    trueVis[group[0]]['yx'] = np.conj(trueVis[group[0]]['xy'])

trueGains = {}
for ant in ha.antNames:
    trueGains[ant] = {pol: 1.0 + .01*(np.random.randn() + np.random.randn()) for pol in antpols}

obsVis, equations = {}, {}
for group in ha.ublGroups: 
    for (i,j) in group:
        obsVis[(i,j)] = {pol: trueGains[i][pol[0]] * np.conj(trueGains[j][pol[1]]) * trueVis[group[0]][pol] for pol in vispols}
        for pol in vispols:
            if pol == 'yx':
                eqStr = 'g{}{} * g{}{}_ * v{}to{}xy_'.format(i,pol[0],j,pol[1],group[0][0],group[0][1])
            else:
                eqStr = 'g{}{} * g{}{}_ * v{}to{}{}'.format(i,pol[0],j,pol[1],group[0][0],group[0][1],pol)
            equations[eqStr] = obsVis[(i,j)][pol]

logSolver = capo.linsolve.LogProductSolver(equations)
logSol = logSolver.solve()
linSolver = capo.linsolve.LinProductSolver(equations, logSol)
linSol = linSolver.solve()

chiSq = 0
for group in ha.ublGroups: 
    for (i,j) in group:
        for pol in vispols:
            if pol == 'yx':
                chiSq += np.abs(obsVis[(i,j)][pol] - linSol['g{}{}'.format(i,pol[0])] * np.conj(linSol['g{}{}'.format(j,pol[1])])
                                * np.conj(linSol['v{}to{}xy'.format(group[0][0], group[0][1])]))**2
            else:
                chiSq += np.abs(obsVis[(i,j)][pol] - linSol['g{}{}'.format(i,pol[0])] * np.conj(linSol['g{}{}'.format(j,pol[1])])
                                * linSol['v{}to{}{}'.format(group[0][0], group[0][1], pol)])**2
                
                
print 'Chisq:', chiSq

A = linSolver.ls.get_A()
A = A.reshape((A.shape[0],A.shape[1]))
AtA = A.T.dot(A)
print 'Number of omnical degeneracies: ', len(AtA) - np.linalg.matrix_rank(AtA)

With 19 antennas there are 30 unique baselines.
Chisq: 1.93803749858e-29
Number of omnical degeneracies:  4
